# Types


## INT

* `TINYINT` (1 byte), `SMALLINT` (2 bytes), `MEDIUMINT` (3 bytes), `INT` (4 bytes), `BIGINT` (8 bytes)

```sql
CREATE TABLE tbl (
    x INT(4) ZEROFILL,
    ...
);
```

* `INT(width)` specifies the display width. 


* `ZEROFILL` replaces space with zero.


## DECIMAL

* Fixed-point type: `DECIMAL`(p, d), where p = precision and d = number of digits after the decimal point

```sql
DECIMAL                 # p=10
DECIMAL(5)              # p=5
DECIMAL(5,2) ZEROFILL    
```

* Floating-point types: `DOUBLE`, `FLOAT`


## BOOLEAN

Use TINYINT(1).


## ENUM

```sql
CREATE TABLE tbl (
    x ENUM('Apple', 'Banana', 'Graph');
    ...
);
```

Note that ENUM is not SQL-standard and not many database system support it.


## CHAR, VARCHAR

CHAR and VARCHAR types use the UTF-8 character set by default. UTF-8 uses up to three bytes to store each character (for example 1 byte for English character and 3 bytes for Korean). 

Assume we use the UTF-8 character set. With CHAR, MySQL reserves three bytes for each character. If we store 'AB' with CHAR, then MySQL uses 6 bytes. With VARCHAR, MySQL reserves one byte for each English character and one byte to store the length of the string. Thus saving 'AB' with VARCHAR needs 3 bytes. So we can save memory by using VARCHAR.

## Large object types (LOB)

* TINYBLOB, BLOB, MEDIUMBLOB, LONGBLOB
* TINYTEXT, TEXT, MEDIUMTEXT, LARGETEXT

# Strings

## Character set, Collation

MySQL's default characterset and collation are latin1 and latin1_swedish_ci.

* Character set

```sql
SHOW CHARACTER SET;

SELECT @@character_set_database;   # Default character set of the database being used

SELECT DEFAULT_CHARACTER_SET_NAME
FROM INFORMATION_SCHEMA.SCHEMATA WHERE SCHEMA_NAME = 'testdb';
```

To set a character set for a client connection, do

```sql
SET NAMES 'utf8'
```

or in the configuration file write

```sql
default-character-set=utf8
```


* Collation

A collation is a set of rules used to compare characters.

```sql
SHOW COLLATION LIKE '%kor%';

SELECT @@collation_database;       # Default collation of the database being used

SELECT DEFAULT_COLLATION_NAME
FROM INFORMATION_SCHEMA.SCHEMATA WHERE SCHEMA_NAME = 'testdb';
```

You can convert the collation of an expression by using COLLATE:
```sql
SELECT ...
FROM ...
WHERE name COLLATE Latin1_General_CS_AS = N'Dave';     --CS: Case-sensitive, AS: Accent-sensitive
```


SQL Server:
```
SELECT name, description
FROM sys.fn_helpcollations();
```


## String functions

### CHAR()

`CHAR()` converts one or more numbers between 0 and 255 to a binary string. It is typically used to insert control characters into a character string.

```sql
CHAR(9)      -- Tab ('\t')
CHAR(10)     -- Line feed ('\n')
CHAR(13)     -- Carriage return ('\r')
CHAR(13,10)  -- Carriage return + Line feed ('\r\n')
```

### SPACE()

`SPACE(count)` returns the space character repeated count times.


### LEFT(), RIGHT(), MID(), SUBSTRING()
   
```sql
LEFT('football',4)	  -- foot
RIGHT('football',4)	  -- ball
MID('football',5)	  -- ball; MID is same as SUBSTRING
MID('football',4,3)	  -- tba
```

### UPPER(), LOWER(), REVERSE()


### TRIM(), RTRIM(), LTRIM()

```sql
TRIM('  Hello   ')                           -- 'Hello'
TRIM(BOTH '*' FROM '**Hello***')             -- 'Hello'
SELECT TRIM(LEADING '*' FROM '**Hello***');  -- 'Hello***'
SELECT TRIM(TRAILING '*' FROM '**Hello***'); -- '**Hello'
```


### SUBSTRING_INDEX(), CHARINDEX(), PATINDEX()

MySQL:

```sql
SUBSTRING_INDEX('testuser:testpass:testdb', ':', 1)      -- testuser
SUBSTRING_INDEX('testuser:testpass:testdb', ':', -1)     -- testdb

SUBSTRING_INDEX(fullname, ' ', 1) AS firstname
SUBSTRING_INDEX(fullname, ' ', -1) AS lastname
```

SQL Server:

```sql
CHARINDEX(' ', 'Hello, guys!')    -- 7
```

`PATINDEX` returns the position of the first occurrence of a pattern within a string:

```sql
PATINDEX('%[0-9]%', 'abc12-de3')  -- 4
```

### INSERT(), REPLACE(), STUFF()

`INSERT`(str, start, length, str_to_insert)

```sql

INSERT('John?', 1, 0, 'How are you, ')      -- How are you, John?
INSERT('John?', 1, 2, 'How are you, ')      -- How are you, hn?


REPLACE('1-a 2-b', '-', ':')      -- 1:a 2:b

REPLACE(RIGHT('(602) 888-5050', 13), ') ', '-')    -- 602-888-5050
```

STUFF(string, pos, delete_length, insert_string)

```sql
STUFF('abc def', 2, 2, 'uvw')      -- auvw def
STUFF('abc def', 2, 3, 'uvw')      -- auvwdef
```

### LOCATE()

```sql
LOCATE('abc', 'barfoobar');      -- 0
LOCATE('bar', 'barfoobar');      -- 1
LOCATE('bar', 'barfoobar', 4);   -- 7

LOCATE(' ', 'John Somang Choi')    -- 5 (first space)
LOCATE(' ', 'John Somang Choi', LOCATE(' ', 'John Somang Choi')+1)    -- 12   (second space)
```


### REPEAT(), REPLICATE()

MySQL:
```sql
REPEAT('*', 3)		-- ***
REPEAT('*', 3.2)    -- ***
REPEAT('*', 3.5)	-- ****
```

SQL Server:
```sql
REPLICATE('*', 3)    -- ***
REPLICATE('*', 3.2)  -- ***
REPLICATE('*', 3.5)  -- ***
REPLICATE('*', 3.8)  -- ***
```


### QUOTE()

```sql
QUOTE("That's it!")      -- 'That\'s it!'
QUOTE('It''s okay.')     -- 'It\'s okay.'
QUOTE(5.3)               -- '5.3'
QUOTE(5.3+4)             -- '9.4'
```

### CONCAT(), CONCAT_WS(), GROUP_CONCAT()
        
```sql
CONCAT('Hi, ', IFNULL(name, 'there'), '!')
```

    WS: With Separator.
```sql
CONCAT('SELECT ', CONCAT_WS(',', 'a', 'b', 'c'))     -- SELECT a,b,c
```

```sql
SELECT student_name,
GROUP_CONCAT(test_score)    -- format: score1,score2,score3
FROM student
GROUP BY student_name;
```

Try with some properties:

```sql
GROUP_CONCAT(DISTINCT test_score ORDER BY test_score DESC SEPARATOR ';')
```


### LPAD(), RPAD()

```sql
LPAD('hi',4,'?')	-- '??hi'
LPAD('hi',1,'?')	-- 'h'
RPAD('hi',4,'?')	-- 'hi??'
```

### LEN(), LENGTH(), CHAR_LENGTH(), DATALENGTH()

MySQL:

* LENGTH(): the length of a string, measured in bytes
* CHAR_LENGTH(): the length of a string, measured in characters
    
```sql
CHAR_LENGTH('하늘')     -- 2 
LENGTH('하늘')          -- 4
```

SQL Server:

* LEN(): the number of characters
* DATALENGTH(): the number of bytes

```sql
LEN(N'ab')           -- 2
DATALENGTH(N'ab')    -- 4
```

### FORMAT()

```sql
FORMAT(312, '0000')     -- 0312
FORMAT(312, 'd5')       -- 00312
FORMAT(312, '07')       -- 3127

FORMAT(232.1682, 2)     -- 232.17
```

### COMPRESS(), DECOMPRESS()

COMPRESS(string) returns a compressed VARBINARY(MAX) typed value.

DECOMPRESS(string) returns a decompressed VARBINARY(MAX) typed value.

# Date/Time

MySQL: `DATE`(3 bytes), `TIME`(3 bytes), `DATETIME`(8 bytes), `TIMESTAMP`(4 bytes), `YEAR[(2|4)]` (1 byte)

* Generally, using `TIMESTAMP` is better than using `DATETIME`, since `TIMESTAMP` requires 4 bytes and `DATETIME` requires 8 bytes. But `TIMESTAMP` can only store dates up to the year 2038. So to store dates beyond 2038, we should use `DATETIME`. 


* A column of `TIMESTAMP` type is automatically updated to the current date/time, when a row is inserted or updated. If a table has multiple TIMESTAMP columns, only the first one is updated automatically.



* `YEAR` is of the form yyyy. `YEAR(2)` is of the form yy.


* The default date format is 'yyyy-mm-dd' and the default time format is 'hh:mm:ss'. Any punctuation character in a literal is considered as a delimiter. The following literal values are all stored as 2010-05-23 in `DATE` column:

```sql
'2010-05-23', '2010-5-23', '10-5-23', '20100523', '2010.05.23', '10/5/23'
```

    But the following is an error:

```sql
'5/23/10'
```

SQL Server: DATETIME, SMALLDATETIME, DATE, TIME, DATETIME2, DATETIMEOFFSET




## LANGUAGE

SQL Server:
```sql
SET LANGUAGE British;
SELECT CAST('02/12/2016' AS DATE);    -- 2016-12-02
SELECT CAST('20160212' AS DATE);      -- 2016-02-12

SET LANGUAGE us_english;
SELECT CAST('02/12/2016' AS DATE);    -- 2016-02-12
SELECT CAST('20160212' AS DATE);      -- 2016-02-12
```


## Timestamp to Date

MySQL: 
```sql
SELECT DATE(NOW());
```

MS SQL: 
```sql
SELECT CONVERT(DATE, CURRENT_TIMESTAMP);
```

## SWITCHOFFSET(), TODATETIMEOFFSET()

SQL Server:

SWITCHOFFSET() adjusts an input DATEITMEOFFSET() value to a specified target offset from UTC.

```sql
SWITCHOFFSET(SYSDATETIMEOFFSET(), '-05:00')

SWITCHOFFSET(SYSDATETIMEOFFSET(), '+00:00')     -- to UTC
```


## TO_DAYS(), TIME_TO_SEC(), STR_TO_DATE()

`TO_DAYS(date)` returns the number of days since the year 0.

`TIME_TO_SEC(time)` returns the number of seconds elapsed since midnight.

`STR_TO_DATE(str, format)` returns a DATE, TIME, or DATETIME value depending on the given format.

```sql
STR_TO_DATE('July 13, 2020', '%M %d, %Y')
STR_TO_DATE('JUL-13-2020', '%b-%d-%Y')
```

## MAKETIME(), TIMESTAMP()

```sql
MAKETIME(10, 30, 0)                         -- 10:30:00
TIMESTAMP('2020-04-15', '10:30:27')         -- 2020-04-15 10:30:27
```

## DATEFROMPARTS(), DATETIME2FROMPARTS(), DATETIMEFROMPARTS(), SMALLDATETIMEFROMPARTS(), TIMEFROMPARTS()

```sql
DATEFROMPARTS(2030, 5, 21)
TIMEFROMPARTS(4, 50, 27, 1, 5)    -- 04:50:27.00001
```


## DATE_FORMAT(), TIME_FORMAT()

```sql
SET @x = TIMESTAMP('2020-04-15', '13:30:27');

SELECT DATE_FORMAT(@x, '%M %d, %Y');       -- April 15, 2020
SELECT DATE_FORMAT(@x, '%Y-%m-01');        -- 2020-04-01
SELECT TIME_FORMAT(@x, '%h');              -- 01
SELECT TIME_FORMAT(@x,'%H h %i m 00 s');   -- 13 h 30 m 00 s
```

Format:

* %y: Year (yy)
* %Y: Year (yyyy)


* %m: Month (01, ..., 12)
* %c: Month (1, ..., 12)
* %M: Month (January, ...)
* %b: Month (Jan, ...)


* %W: Weekday (Sunday, ...)
* %a: Weekday (Sun, ...)

* %d: Day (00, ..., 31)
* %e: Day (0, ..., 31)
* %D: Day (1st, ... )


* %H: Hour (00, ..., 23)
* %k: Hou (0, .... 23)
* %h: Hour (01, ..., 12)
* %l: Hour (1, ..., 12)


* %i: Minutes (00, ..., 59)
* %S: Seconds (00, ..., 59)


* %p: AM or PM
* %r: Time, 12-hr (hh:mm:ss AM|PM)
* %T: Time, 24-hr (hh:mm:ss)


## Current Date/Time

* `CURRENT_DATE`, `CURRENT_DATE()`, `CURDATE()`


* `CURRENT_TIME`, `CURRENT_TIME()`, `CURTIME()`


* `CURRENT_TIMESTAMP`, `CURRENT_TIMESTAMP()`, `NOW()`, `SYSDATE()`


* `UTC_DATE()`, `UTC_TIME()`


* `GETDATE()`, `GETUTCDATE()`, `SYSDATETIME()`, `SYSUTCDATETIME()`, `SYSDATETIMEOFFSET()` (SQL Server)


## Parts of Date/Time

### YEAR(), QUARTER(), MONTH(), WEEK(), DAY()

```sql
SELECT name, birth, MONTH(birth) FROM pet;
SELECT name, birth FROM pet WHERE MONTH(birth) = 5;
```

### DAYNAME(), DATENAME(), MONTHNAME()

```sql
DATENAME(month, '20200405')
```

### DAYOFMONTH(), DAYOFWEEK(), WEEKDAY(), DAYOFYEAR()

### HOUR(), MINUTE(), SECOND()

### LAST_DAY(), EOMONTH()

* LAST_DAY() in MySQL
* EOMONTH() in SQL Server


### EXTRACT(), DATEPART()

MySQL:

`EXTRACT(unit FROM date)`, where unit is SECOND, MINUTE, HOUR, DAY, MONTH, YEAR, MINUTE_SECOND, HOUR_MINUTE, HOUR_SECOND (hour, min, sec), DAY_HOUR, DAY_MINUTE (day, hour, min), DAY_SECOND (day, hour, min, sec), YEAR_MONTH.


```sql
EXTRACT(YEAR FROM '2020-07-14 11:43:55')                  -- 2020
EXTRACT(YEAR_MONTH FROM '2020-07-14 11:43:55')            -- 202007
EXTRACT(HOUR_MINUTE FROM '2020-07-14 11:43:55')           -- 1143
EXTRACT(DAY_MINUTE FROM '2020-07-14 11:43:55')            -- 141143
```

SQL Server:

```sql
DATEPART(month, '20160512')
```

## Timezones

### time_zone, mysql.time_zone_name

```sql
SELECT * FROM mysql.time_zone_name;

SELECT @@global.time_zone, @@session.time_zone;
SET time_zone = 'Asia/Seoul';
```

### CONVERT_TZ(dt,from_tz,to_tz)

```sql
CONVERT_TZ('2020-11-28 17:50:00', 'Asia/Seoul', 'UTC')
```

### AT TIME ZONE

SQL Server:

```sql
SELECT CAST('20160212 12:00:00.0000000' AS DATETIME2)
AT TIME ZONE 'Pacific Standard Time';
```



## Date/Time operations

### DATE_ADD(), DATE_SUB(), ADDTIME(), INTERVAL

```sql
SELECT name, birth FROM pet
WHERE MONTH(birth) = MONTH(DATE_ADD(CURDATE(),INTERVAL 1 MONTH));

SELECT NOW() + INTERVAL 1 DAY;
       
DATE_ADD(CURDATE(), INTERVAL 3 DAY)
CURDATE() + INTERVAL 3 DAY

DATE_ADD(NOW(), INTERVAL '13:20:30' HOUR_SECOND)      # '13 20 30', '13-20-30', ... also work.
NOW() + INTERVAL '13:20:30' HOUR_SECOND

DATE_ADD(NOW(), INTERVAL '10-6' YEAR_MONTH)           # '10 6', '10---6', ... also work.
NOW() + INTERVAL '10-6' YEAR_MONTH 

DATE_SUB(CURDATE(), INTERVAL 2 WEEK)                  
CURDATE() - INTERVAL 2 WEEK

ADDTIME('10:37:58', '15:39:14')
ADDTIME(NOW(), '15:39:14')
```

### DATEDIFF(), TIMEDIFF(), TIMESTAMPDIFF()

* DATEDIFF(dt1, dt2) returns dt1 - dt2 in days.
* The result returned by TIMEDIFF() is limited to the range allowed for TIME values.
* TIMESTAMPDIFF(unit, dt1, dt2) returns dt2 - dt1 in unit.

```sql
SELECT name, birth, CURDATE(), TIMESTAMPDIFF(YEAR,birth,CURDATE()) AS age FROM pet;

```

### DATEADD(), DATEDIFF(), DATEDIFF_BIG() in SQL Server

```sql
DATEADD(interval, number, date)        # date + number
DATEDIFF(interval, date1, date2)       # data2 - data1

SELECT DATEADD(month, 7, '2018/05/20');      # 2018-12-20 00:00:00.000
SELECT DATEADD(month, -7, '2018/05/20');     # 2017-10-20 00:00:00.000

SELECT DATEDIFF(month, '2019/04/25', '2018/05/20');   # -11
SELECT DATEDIFF(month, '2018/05/20', '2019/04/25');   # 11
```


# Functions related to Data types


## CONVERT(), CAST(), PARSE()

My SQL:

* CONVERT(expr, type) = CAST(expr AS type)
* CONVERT(expr USING characterset_name)
* CAST(expr AS data_type CHARACTER SET characterset_name)
 
 
```sql
CONVERT('abc' USING utf8mb4)

CAST('2019-05-30' AS DATE)

CAST(_latin1'abc' AS CHAR CHARACTER SET utf8);
```

CAST(str AS [DATE|TIME|DATETIME])

```sql
CAST(col1 AS NUMERIC(10,2))/CAST(col2 AS NUMERIC(10,2))


CAST('2020-07-13' AS DATE)
CAST('2020-07-13 14:20:00' AS DATETIME)

CAST('5.2xyz' AS UNSIGNED)       # 5
CAST('5.2xyz' AS DOUBLE)         # 5.2
CAST('5.2xyz' AS INT)            # Error 

SELECT '5.2xyz'+0                # 5.2

CAST('xy5' AS UNSIGNED)       # 0
CAST('xy5' AS DOUBLE)         # 0
CAST('xy5' AS INT)            # Error 

SELECT 'xy5'+0                # 0
```

SQL Server:
```sql
CAST(SYSDATETIME() AS DATE) AS [current date]
CAST(SYSDATETIME() AS TIME) AS [current date]
```

SQL Server:
```sql
SELECT CONVERT(DECIMAL(10,2), 10.2284);          -- 10.23
SELECT ROUND(10.2284, 2);                        -- 10.2300

CONVERT(DATE, '02/12/2016', 101)   -- 2016-02-12; style 101 is US English
CONVERT(DATE, '02/12/2016', 103)   -- 2016-12-02; style 103 is British English
```

```sql
PARSE('02/12/2016' AS DATE USING 'en-GB')  -- 2016-12-02
PARSE('02/12/2016' AS DATE USING 'en-US')  -- 2016-02-12
```

## TRY_CONVERT(), TRY_CAST(), TRY_PARSE() (SQL Server)

If the input cannot be converted, try_ functions returns NULL values.

## ISDATE() (SQL Server)

```sql
ISDATE('20201230')
```

# Spatial data types

GEOMETRY, POINT, LINESTRING, POLYGON, GEOMETRYCOLLECTION, MULTILINESTRING, MULTIPOINT, MULTIPOLYGON